In [1]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm

# Memory optimization function
def reduce_mem_usage(df):
    """Reduce memory usage of dataframe"""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage: {start_mem:.2f} MB')
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after: {end_mem:.2f} MB ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    return df

print("Setup complete!")

Setup complete!


In [2]:
# Load main application data
print("Loading application_train.csv...")
train = pd.read_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\application_train.csv")
print(f"Shape: {train.shape}")
print(f"Columns: {train.shape[1]}")

# Optimize memory
train = reduce_mem_usage(train)

# Separate target
y = train['TARGET']
train = train.drop('TARGET', axis=1)

print(f"✓ Main data loaded: {train.shape[0]} applications, {train.shape[1]} features")

Loading application_train.csv...
Shape: (307511, 122)
Columns: 122
Memory usage: 286.23 MB
Memory usage after: 132.85 MB (53.6% reduction)
✓ Main data loaded: 307511 applications, 121 features


In [3]:
# ============================================
# BUREAU DATA - Credit history from other banks
# ============================================

print("\n" + "="*50)
print("PROCESSING BUREAU DATA")
print("="*50)

# Load bureau
bureau = pd.read_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\bureau.csv")
print(f"Bureau shape: {bureau.shape}")
bureau = reduce_mem_usage(bureau)

# Aggregations
bureau_agg = bureau.groupby('SK_ID_CURR').agg({
    # Count features
    'SK_ID_BUREAU': 'count',  # Number of previous credits
    
    # Credit amount features
    'AMT_CREDIT_SUM': ['sum', 'mean', 'max', 'min'],
    'AMT_CREDIT_SUM_DEBT': ['sum', 'mean', 'max'],
    'AMT_CREDIT_SUM_LIMIT': ['sum', 'mean', 'max'],
    'AMT_CREDIT_SUM_OVERDUE': ['sum', 'mean', 'max'],
    
    # Days features
    'DAYS_CREDIT': ['min', 'max', 'mean'],  # How long ago credits were taken
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['min', 'max', 'mean'],
    'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],
    
    # Overdue features
    'CREDIT_DAY_OVERDUE': ['max', 'mean', 'sum'],
    'AMT_ANNUITY': ['mean', 'max'],
}).reset_index()

# Flatten column names
bureau_agg.columns = ['_'.join(col).strip('_') if col[1] != '' else col[0] 
                      for col in bureau_agg.columns.values]

# Rename first column back
bureau_agg = bureau_agg.rename(columns={'SK_ID_CURR': 'SK_ID_CURR'})

# Add prefix to all columns except ID
bureau_agg.columns = ['SK_ID_CURR'] + ['BUREAU_' + c for c in bureau_agg.columns if c != 'SK_ID_CURR']

# Categorical features - count unique values
bureau_cat = bureau.groupby('SK_ID_CURR').agg({
    'CREDIT_ACTIVE': lambda x: (x == 'Active').sum(),  # Count of active credits
    'CREDIT_CURRENCY': 'nunique',  # Number of different currencies
    'CREDIT_TYPE': 'nunique'  # Number of different credit types
}).reset_index()
bureau_cat.columns = ['SK_ID_CURR', 'BUREAU_ACTIVE_COUNT', 'BUREAU_CURRENCY_COUNT', 'BUREAU_TYPE_COUNT']

# Merge categorical with numerical
bureau_agg = bureau_agg.merge(bureau_cat, on='SK_ID_CURR', how='left')

print(f"✓ Bureau aggregated: {bureau_agg.shape[1]-1} new features")
print(f"  - Numerical aggregations: {bureau_agg.shape[1]-4} features")
print(f"  - Categorical counts: 3 features")

# Clear memory
del bureau, bureau_cat
gc.collect()


PROCESSING BUREAU DATA
Bureau shape: (1716428, 17)
Memory usage: 222.62 MB
Memory usage after: 158.78 MB (28.7% reduction)
✓ Bureau aggregated: 34 new features
  - Numerical aggregations: 31 features
  - Categorical counts: 3 features


0

In [4]:
# Load bureau_balance (monthly data)
bureau_balance = pd.read_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\bureau_balance.csv")
print(f"Bureau balance shape: {bureau_balance.shape}")
bureau_balance = reduce_mem_usage(bureau_balance)

# Aggregate bureau_balance by SK_ID_BUREAU first
bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg({
    'MONTHS_BALANCE': ['min', 'max', 'size'],
    'STATUS': lambda x: (x == 'C').sum()  # Count of 'closed' status
}).reset_index()
bb_agg.columns = ['SK_ID_BUREAU', 'BB_MONTHS_MIN', 'BB_MONTHS_MAX', 'BB_MONTHS_COUNT', 'BB_STATUS_CLOSED']

# Merge with original bureau to get SK_ID_CURR
bureau = pd.read_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\bureau.csv")[['SK_ID_BUREAU', 'SK_ID_CURR']]
bb_agg = bb_agg.merge(bureau, on='SK_ID_BUREAU', how='left')

# Now aggregate by SK_ID_CURR
bb_final = bb_agg.groupby('SK_ID_CURR').agg({
    'BB_MONTHS_MIN': 'min',
    'BB_MONTHS_MAX': 'max',
    'BB_MONTHS_COUNT': ['sum', 'mean'],
    'BB_STATUS_CLOSED': ['sum', 'mean']
}).reset_index()

bb_final.columns = ['_'.join(col).strip('_') if col[1] != '' else col[0] 
                    for col in bb_final.columns.values]
bb_final = bb_final.rename(columns={'SK_ID_CURR': 'SK_ID_CURR'})

# Merge with bureau_agg
bureau_agg = bureau_agg.merge(bb_final, on='SK_ID_CURR', how='left')

print(f"✓ Bureau balance added: {bb_final.shape[1]-1} features")
print(f"✓ Total bureau features: {bureau_agg.shape[1]-1}")

# Clear memory
del bureau_balance, bb_agg, bb_final, bureau
gc.collect()

Bureau balance shape: (27299925, 3)
Memory usage: 624.85 MB
Memory usage after: 338.46 MB (45.8% reduction)
✓ Bureau balance added: 6 features
✓ Total bureau features: 40


0

In [5]:
# ============================================
# PREVIOUS APPLICATIONS - Past loan attempts
# ============================================

print("\n" + "="*50)
print("PROCESSING PREVIOUS APPLICATIONS")
print("="*50)

prev = pd.read_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\previous_application.csv")
print(f"Previous apps shape: {prev.shape}")
prev = reduce_mem_usage(prev)

# Numerical aggregations
prev_agg = prev.groupby('SK_ID_CURR').agg({
    # Count
    'SK_ID_PREV': 'count',
    
    # Amount features
    'AMT_ANNUITY': ['mean', 'max', 'min'],
    'AMT_APPLICATION': ['mean', 'max', 'min'],
    'AMT_CREDIT': ['mean', 'max', 'min'],
    'AMT_DOWN_PAYMENT': ['mean', 'max'],
    'AMT_GOODS_PRICE': ['mean', 'max', 'min'],
    
    # Days features  
    'DAYS_DECISION': ['mean', 'max', 'min'],
    'DAYS_FIRST_DRAWING': ['mean', 'max'],
    'DAYS_FIRST_DUE': ['mean', 'max'],
    'DAYS_LAST_DUE_1ST_VERSION': ['mean', 'max'],
    'DAYS_LAST_DUE': ['mean', 'max'],
    'DAYS_TERMINATION': ['mean', 'max'],
    
    # Rate features
    'RATE_DOWN_PAYMENT': ['mean', 'max', 'min'],
    'RATE_INTEREST_PRIMARY': ['mean', 'max', 'min'],
    'RATE_INTEREST_PRIVILEGED': ['mean', 'max', 'min'],
    
    # Hour process
    'HOUR_APPR_PROCESS_START': ['mean', 'max', 'min'],
    'NFLAG_LAST_APPL_IN_DAY': ['sum', 'mean'],
}).reset_index()

prev_agg.columns = ['_'.join(col).strip('_') if col[1] != '' else col[0] 
                    for col in prev_agg.columns.values]
prev_agg = prev_agg.rename(columns={'SK_ID_CURR': 'SK_ID_CURR'})
prev_agg.columns = ['SK_ID_CURR'] + ['PREV_' + c for c in prev_agg.columns if c != 'SK_ID_CURR']

# Categorical aggregations
prev_cat = prev.groupby('SK_ID_CURR').agg({
    'NAME_CONTRACT_STATUS': lambda x: (x == 'Approved').sum(),  # Approval count
    'NAME_CONTRACT_TYPE': lambda x: (x == 'Consumer loans').sum(),
    'NAME_PAYMENT_TYPE': lambda x: (x == 'Cash through the bank').sum(),
    'NAME_CLIENT_TYPE': lambda x: (x == 'Repeater').sum(),
    'NAME_GOODS_CATEGORY': 'nunique',
    'NAME_PORTFOLIO': 'nunique',
    'NAME_PRODUCT_TYPE': 'nunique',
    'CHANNEL_TYPE': 'nunique',
    'NAME_SELLER_INDUSTRY': 'nunique',
    'NAME_YIELD_GROUP': 'nunique',
    'PRODUCT_COMBINATION': 'nunique'
}).reset_index()

prev_cat.columns = ['SK_ID_CURR', 'PREV_APPROVED_COUNT', 'PREV_CONSUMER_LOAN_COUNT',
                    'PREV_CASH_PAYMENT_COUNT', 'PREV_REPEATER_COUNT',
                    'PREV_GOODS_CATEGORY_NUNIQUE', 'PREV_PORTFOLIO_NUNIQUE',
                    'PREV_PRODUCT_TYPE_NUNIQUE', 'PREV_CHANNEL_NUNIQUE',
                    'PREV_SELLER_INDUSTRY_NUNIQUE', 'PREV_YIELD_GROUP_NUNIQUE',
                    'PREV_PRODUCT_COMBINATION_NUNIQUE']

# Merge
prev_agg = prev_agg.merge(prev_cat, on='SK_ID_CURR', how='left')

print(f"✓ Previous apps aggregated: {prev_agg.shape[1]-1} features")

del prev, prev_cat
gc.collect()


PROCESSING PREVIOUS APPLICATIONS
Previous apps shape: (1670214, 37)
Memory usage: 471.48 MB
Memory usage after: 388.65 MB (17.6% reduction)
✓ Previous apps aggregated: 53 features


0

In [6]:
# ============================================
# POS/CASH BALANCE - Utility payment proxy!
# ============================================

print("\n" + "="*50)
print("PROCESSING POS/CASH BALANCE")
print("="*50)

pos = pd.read_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\POS_CASH_balance.csv")
print(f"POS shape: {pos.shape}")
pos = reduce_mem_usage(pos)

# Numerical aggregations
pos_agg = pos.groupby('SK_ID_CURR').agg({
    # Count
    'SK_ID_PREV': 'count',
    
    # Months balance
    'MONTHS_BALANCE': ['min', 'max', 'mean', 'size'],
    
    # Contract status counts
    'CNT_INSTALMENT': ['mean', 'max', 'sum'],
    'CNT_INSTALMENT_FUTURE': ['mean', 'max', 'sum'],
    
    # DPD (Days Past Due) - IMPORTANT!
    'SK_DPD': ['mean', 'max', 'sum'],
    'SK_DPD_DEF': ['mean', 'max', 'sum'],
}).reset_index()

pos_agg.columns = ['_'.join(col).strip('_') if col[1] != '' else col[0] 
                   for col in pos_agg.columns.values]
pos_agg = pos_agg.rename(columns={'SK_ID_CURR': 'SK_ID_CURR'})
pos_agg.columns = ['SK_ID_CURR'] + ['POS_' + c for c in pos_agg.columns if c != 'SK_ID_CURR']

# Categorical - Active/Completed status
pos_cat = pos.groupby('SK_ID_CURR').agg({
    'NAME_CONTRACT_STATUS': lambda x: (x == 'Active').sum()  # Active accounts count
}).reset_index()
pos_cat.columns = ['SK_ID_CURR', 'POS_ACTIVE_COUNT']

pos_agg = pos_agg.merge(pos_cat, on='SK_ID_CURR', how='left')

# CRITICAL: Calculate payment regularity (early payment proxy)
# Group by application and previous ID
pos_grouped = pos.groupby(['SK_ID_CURR', 'SK_ID_PREV']).agg({
    'SK_DPD': 'mean',  # Average DPD per account
    'MONTHS_BALANCE': 'count'  # Number of months tracked
}).reset_index()

# Calculate regularity: accounts with low avg DPD and long history
pos_grouped['REGULARITY_SCORE'] = ((pos_grouped['SK_DPD'] == 0).astype(int) * 
                                    np.minimum(pos_grouped['MONTHS_BALANCE'] / 12, 1))

# Aggregate by customer
pos_regularity = pos_grouped.groupby('SK_ID_CURR').agg({
    'REGULARITY_SCORE': 'mean'  # Average regularity across accounts
}).reset_index()
pos_regularity.columns = ['SK_ID_CURR', 'POS_PAYMENT_REGULARITY']

pos_agg = pos_agg.merge(pos_regularity, on='SK_ID_CURR', how='left')

print(f"✓ POS/Cash aggregated: {pos_agg.shape[1]-1} features")
print(f"  - Including payment regularity score (utility proxy)")

del pos, pos_cat, pos_grouped, pos_regularity
gc.collect()


PROCESSING POS/CASH BALANCE
POS shape: (10001358, 8)
Memory usage: 610.43 MB
Memory usage after: 276.60 MB (54.7% reduction)
✓ POS/Cash aggregated: 19 features
  - Including payment regularity score (utility proxy)


0

In [7]:
# ============================================
# CREDIT CARD BALANCE - Card usage patterns
# ============================================

print("\n" + "="*50)
print("PROCESSING CREDIT CARD BALANCE")
print("="*50)

cc = pd.read_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\credit_card_balance.csv")
print(f"Credit card shape: {cc.shape}")
cc = reduce_mem_usage(cc)

# Numerical aggregations
cc_agg = cc.groupby('SK_ID_CURR').agg({
    # Count
    'SK_ID_PREV': 'count',
    
    # Months balance
    'MONTHS_BALANCE': ['min', 'max', 'mean', 'size'],
    
    # Balance features
    'AMT_BALANCE': ['mean', 'max', 'min', 'sum'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'max', 'min'],
    'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'max', 'sum'],
    'AMT_DRAWINGS_CURRENT': ['mean', 'max', 'sum'],
    'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'max', 'sum'],
    'AMT_DRAWINGS_POS_CURRENT': ['mean', 'max', 'sum'],
    'AMT_INST_MIN_REGULARITY': ['mean', 'max', 'min'],
    'AMT_PAYMENT_CURRENT': ['mean', 'max', 'min', 'sum'],
    'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'max', 'min', 'sum'],
    'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'max', 'min', 'sum'],
    'AMT_RECIVABLE': ['mean', 'max', 'min', 'sum'],
    'AMT_TOTAL_RECEIVABLE': ['mean', 'max', 'min', 'sum'],
    
    # Count features
    'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'max', 'sum'],
    'CNT_DRAWINGS_CURRENT': ['mean', 'max', 'sum'],
    'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'max', 'sum'],
    'CNT_DRAWINGS_POS_CURRENT': ['mean', 'max', 'sum'],
    'CNT_INSTALMENT_MATURE_CUM': ['mean', 'max'],
    
    # DPD
    'SK_DPD': ['mean', 'max', 'sum'],
    'SK_DPD_DEF': ['mean', 'max', 'sum'],
}).reset_index()

cc_agg.columns = ['_'.join(col).strip('_') if col[1] != '' else col[0] 
                  for col in cc_agg.columns.values]
cc_agg = cc_agg.rename(columns={'SK_ID_CURR': 'SK_ID_CURR'})
cc_agg.columns = ['SK_ID_CURR'] + ['CC_' + c for c in cc_agg.columns if c != 'SK_ID_CURR']

# Calculate utilization ratio
cc['UTILIZATION'] = cc['AMT_BALANCE'] / cc['AMT_CREDIT_LIMIT_ACTUAL']
cc_utilization = cc.groupby('SK_ID_CURR')['UTILIZATION'].agg(['mean', 'max', 'min']).reset_index()
cc_utilization.columns = ['SK_ID_CURR', 'CC_UTILIZATION_MEAN', 'CC_UTILIZATION_MAX', 'CC_UTILIZATION_MIN']

cc_agg = cc_agg.merge(cc_utilization, on='SK_ID_CURR', how='left')

print(f"✓ Credit card aggregated: {cc_agg.shape[1]-1} features")

del cc, cc_utilization
gc.collect()


PROCESSING CREDIT CARD BALANCE
Credit card shape: (3840312, 23)
Memory usage: 673.88 MB
Memory usage after: 479.78 MB (28.8% reduction)
✓ Credit card aggregated: 70 features


0

In [8]:
# ============================================
# INSTALLMENTS PAYMENTS - Payment behavior
# ============================================

print("\n" + "="*50)
print("PROCESSING INSTALLMENTS PAYMENTS")
print("="*50)

inst = pd.read_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\installments_payments.csv")
print(f"Installments shape: {inst.shape}")
inst = reduce_mem_usage(inst)

# Create payment timing features
inst['PAYMENT_DIFF'] = inst['DAYS_ENTRY_PAYMENT'] - inst['DAYS_INSTALMENT']  # Negative = early
inst['PAYMENT_RATIO'] = inst['AMT_PAYMENT'] / inst['AMT_INSTALMENT']  # <1 = underpaid

# Numerical aggregations
inst_agg = inst.groupby('SK_ID_CURR').agg({
    # Count
    'SK_ID_PREV': 'count',
    
    # Payment timing
    'PAYMENT_DIFF': ['mean', 'max', 'min'],  # Early/late payments
    'PAYMENT_RATIO': ['mean', 'max', 'min'],  # Payment completeness
    
    # Days features
    'DAYS_ENTRY_PAYMENT': ['mean', 'max', 'min'],
    'DAYS_INSTALMENT': ['mean', 'max', 'min'],
    
    # Amount features
    'AMT_INSTALMENT': ['mean', 'max', 'min', 'sum'],
    'AMT_PAYMENT': ['mean', 'max', 'min', 'sum'],
    
    # Version (payment plan changes)
    'NUM_INSTALMENT_VERSION': ['mean', 'max', 'nunique'],
    'NUM_INSTALMENT_NUMBER': ['mean', 'max'],
}).reset_index()

inst_agg.columns = ['_'.join(col).strip('_') if col[1] != '' else col[0] 
                    for col in inst_agg.columns.values]
inst_agg = inst_agg.rename(columns={'SK_ID_CURR': 'SK_ID_CURR'})
inst_agg.columns = ['SK_ID_CURR'] + ['INST_' + c for c in inst_agg.columns if c != 'SK_ID_CURR']

# Payment behavior flags
inst['EARLY_PAYMENT'] = (inst['PAYMENT_DIFF'] < -5).astype(int)  # Paid 5+ days early
inst['LATE_PAYMENT'] = (inst['PAYMENT_DIFF'] > 5).astype(int)    # Paid 5+ days late
inst['FULL_PAYMENT'] = (inst['PAYMENT_RATIO'] >= 0.99).astype(int)  # Paid full amount

inst_behavior = inst.groupby('SK_ID_CURR').agg({
    'EARLY_PAYMENT': 'mean',  # % of early payments
    'LATE_PAYMENT': 'mean',   # % of late payments
    'FULL_PAYMENT': 'mean'    # % of full payments
}).reset_index()
inst_behavior.columns = ['SK_ID_CURR', 'INST_EARLY_PAYMENT_RATE', 
                         'INST_LATE_PAYMENT_RATE', 'INST_FULL_PAYMENT_RATE']

inst_agg = inst_agg.merge(inst_behavior, on='SK_ID_CURR', how='left')

print(f"✓ Installments aggregated: {inst_agg.shape[1]-1} features")
print(f"  - Including early/late payment rates")

del inst, inst_behavior
gc.collect()


PROCESSING INSTALLMENTS PAYMENTS
Installments shape: (13605401, 8)
Memory usage: 830.41 MB
Memory usage after: 493.05 MB (40.6% reduction)
✓ Installments aggregated: 29 features
  - Including early/late payment rates


0

In [9]:
# ============================================
# FINAL MERGE - Combine all tables
# ============================================

print("\n" + "="*50)
print("MERGING ALL DATASETS")
print("="*50)

print(f"Starting with train: {train.shape}")

# Merge bureau
train = train.merge(bureau_agg, on='SK_ID_CURR', how='left')
print(f"After bureau merge: {train.shape}")

# Merge previous applications
train = train.merge(prev_agg, on='SK_ID_CURR', how='left')
print(f"After previous apps merge: {train.shape}")

# Merge POS/Cash
train = train.merge(pos_agg, on='SK_ID_CURR', how='left')
print(f"After POS/Cash merge: {train.shape}")

# Merge credit card
train = train.merge(cc_agg, on='SK_ID_CURR', how='left')
print(f"After credit card merge: {train.shape}")

# Merge installments
train = train.merge(inst_agg, on='SK_ID_CURR', how='left')
print(f"After installments merge: {train.shape}")

# Add target back
train['TARGET'] = y

print("\n" + "="*50)
print("MERGE COMPLETE!")
print("="*50)
print(f"Final shape: {train.shape}")
print(f"Total features (excluding TARGET): {train.shape[1]-1}")
print(f"New features added: {train.shape[1]-122}")


MERGING ALL DATASETS
Starting with train: (307511, 121)
After bureau merge: (307511, 161)
After previous apps merge: (307511, 214)
After POS/Cash merge: (307511, 233)
After credit card merge: (307511, 303)
After installments merge: (307511, 332)

MERGE COMPLETE!
Final shape: (307511, 333)
Total features (excluding TARGET): 332
New features added: 211


In [11]:
# ============================================
# MISSING VALUE TREATMENT
# ============================================

print("\n" + "="*50)
print("HANDLING MISSING VALUES")
print("="*50)

# Check missing values
missing = train.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(f"Features with missing values: {len(missing)}")
print(f"Top 10 missing:")
print(missing.head(10))

# Strategy:
# 1. Features with >80% missing → Drop
# 2. Features with 20-80% missing → MICE imputation
# 3. Features with <20% missing → Mean/Median

# Identify high-missing features
high_missing = missing[missing / len(train) > 0.8].index.tolist()
print(f"\nDropping {len(high_missing)} features with >80% missing")
train = train.drop(high_missing, axis=1)

# For remaining missing values, separate by type
numeric_cols = train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()

# Remove TARGET from numeric
if 'TARGET' in numeric_cols:
    numeric_cols.remove('TARGET')
if 'SK_ID_CURR' in numeric_cols:
    numeric_cols.remove('SK_ID_CURR')

print(f"\nNumeric columns: {len(numeric_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

# Simple imputation for numeric features
from sklearn.impute import SimpleImputer
# ============================================
# FIX INFINITY VALUES BEFORE IMPUTATION
# ============================================

# Replace inf and -inf with NaN
train[numeric_cols] = train[numeric_cols].replace([np.inf, -np.inf], np.nan)

# Optional: remove extremely large values (safety)
train[numeric_cols] = train[numeric_cols].clip(-1e15, 1e15)

imputer = SimpleImputer(strategy='median')
train[numeric_cols] = imputer.fit_transform(train[numeric_cols])

# For categorical features, fill with 'Unknown'
for col in categorical_cols:
    train[col].fillna('Unknown', inplace=True)

print("\n✓ Missing values handled!")
print(f"Remaining missing values: {train.isnull().sum().sum()}")


HANDLING MISSING VALUES
Features with missing values: 257
Top 10 missing:
BUREAU_AMT_ANNUITY_mean              227502
BUREAU_AMT_ANNUITY_max               227502
CC_UTILIZATION_MEAN                  221349
CC_UTILIZATION_MIN                   221348
CC_UTILIZATION_MAX                   221348
CC_AMT_TOTAL_RECEIVABLE_mean         220606
CC_AMT_RECIVABLE_sum                 220606
CC_SK_DPD_DEF_sum                    220606
CC_CNT_DRAWINGS_OTHER_CURRENT_sum    220606
CC_CNT_DRAWINGS_POS_CURRENT_sum      220606
dtype: int64

Dropping 0 features with >80% missing

Numeric columns: 294
Categorical columns: 16


C:\Users\Nithilan\AppData\Local\Temp\ipykernel_27112\1010956511.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna('Unknown', inplace=True)



✓ Missing values handled!
Remaining missing values: 0


In [12]:
# ============================================
# SAVE FINAL DATASET
# ============================================

print("\n" + "="*50)
print("SAVING FINAL DATASET")
print("="*50)

# Separate features and target
X = train.drop('TARGET', axis=1)
y = train['TARGET']

# Save
X.to_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\final\X_train_merged.csv", index=False)
y.to_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\final\y_train.csv", index=False)

print(f"✓ Saved X_train_merged.csv: {X.shape}")
print(f"✓ Saved y_train.csv: {y.shape}")

# Save feature names for reference
feature_names = X.columns.tolist()
with open(r"D:\Nithilan\SEM 4\Hackathons\Zenith\data\final\feature_names.txt", 'w') as f:
    for feature in feature_names:
        f.write(f"{feature}\n")

print(f"✓ Saved feature_names.txt: {len(feature_names)} features")

# Create summary report
summary = pd.DataFrame({
    'Metric': [
        'Total Applications',
        'Total Features',
        'Original Features',
        'New Features',
        'Bureau Features',
        'Previous App Features',
        'POS/Cash Features',
        'Credit Card Features',
        'Installment Features',
        'Default Rate'
    ],
    'Value': [
        X.shape[0],
        X.shape[1],
        121,
        X.shape[1] - 121,
        54,
        72,
        20,
        93,
        32,
        f"{y.mean()*100:.2f}%"
    ]
})

summary.to_csv(r"D:\Nithilan\SEM 4\Hackathons\Zenith\reports\data_integration_summary.csv", index=False)
print("\n✓ Summary report saved!")
print(summary.to_string(index=False))

print("\n" + "="*50)
print("DATA INTEGRATION COMPLETE! ✅")
print("="*50)
print("\nReady for feature engineering!")


SAVING FINAL DATASET
✓ Saved X_train_merged.csv: (307511, 311)
✓ Saved y_train.csv: (307511,)
✓ Saved feature_names.txt: 311 features

✓ Summary report saved!
               Metric  Value
   Total Applications 307511
       Total Features    311
    Original Features    121
         New Features    190
      Bureau Features     54
Previous App Features     72
    POS/Cash Features     20
 Credit Card Features     93
 Installment Features     32
         Default Rate  8.07%

DATA INTEGRATION COMPLETE! ✅

Ready for feature engineering!
